# Analysing SEO with Google Analytics

This report explores how we can pull in website data from Google Analytics and find insights on SEO. If you want to run this notebook, the install instructions are kept [in the Readme.md](/Readme.md).

## Introduction

Google Analytics is a website traffic analysis application that provides real-time statistics and analysis of user interaction with the website. Google analytics enables website owners to analyze their visitors, with the objective of interpreting and optimizing website's performance.

In this report we will learn how to pull in the data using the Google Analytics Reporting API and graph the results to show meaningful insights.

## Setting Up Google Analytics Permissions

For reference, Google has a great [tutorial](https://developers.google.com/analytics/devguides/reporting/core/v4/quickstart/service-py) that can take you through cononecting the API, but we're go through it in detail in this report.

#### Enable API and generate a private key

- To get started using Analytics Reporting API v4, you need to first [use the setup tool](https://console.developers.google.com/start/api?id=analyticsreporting.googleapis.com&credential=client_key), which guides you through creating a project in the Google API Console, enabling the API, and creating credentials.

##### Create Credentials
- Open the [Service accounts](https://console.cloud.google.com/iam-admin/serviceaccounts) page. If prompted, select a project.
- Click **+ Create Service Account**, enter a name and description for the service account. You can use the default service account ID, or choose a different, unique one. When done click **Create**.
- The Service account permissions (optional) section that follows is not required. Click **Continue**.
- On the **Grant users access to this service account** screen, scroll down to the **Create key** section. Click **+ Create key**.
- In the side panel that appears, select the format for your key: **JSON** is recommended.
- Click **Create**. Your new public/private key pair is generated and downloaded to your machine; it serves as the only copy of this key. Save this key into the this directory folder and name it **client_secrets.json**.
- Click **Close** on the **Private key saved to your computer** dialog, then click **Done** to return to the table of your service accounts.

##### Add service account to the Google Analytics account

The newly created service account will have an email address that looks similar to:

quickstart@PROJECT-ID.iam.gserviceaccount.com

Use this email address to [add a user](https://support.google.com/analytics/answer/1009702) to the Google analytics view you want to access via the API. For this tutorial only [Read & Analyze](https://support.google.com/analytics/answer/2884495) permissions are needed.

##### Required Installations:
- pip install --upgrade google-api-python-client
- pip install --upgrade oauth2client
- pip install plotly==4.6.0
- pip install cufflinks
- pip install httplib2

In [2]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import httplib2
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
from cufflinks import tools


## Create Variables to Access the Data

Replace the VIEW_ID value. You can use the [Account Explorer](https://ga-dev-tools.appspot.com/account-explorer/) to find a View ID.

In [3]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = 'XXXXXX'

## Define Functions to Pull the Data

In [4]:
def initialize_analyticsreporting():
    """Initializes an Analytics Reporting API V4 service object.

    Returns:
    An authorized Analytics Reporting API V4 service object.
    """
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics


In [5]:
def get_report(analytics,request):
    """Queries the Analytics Reporting API V4.

    Args:
    analytics: An authorized Analytics Reporting API V4 service object.
    Returns:
    The Analytics Reporting API V4 response.
    """
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        request
        ]
      }
    ).execute()

In [6]:
def main(request):
    analytics = initialize_analyticsreporting()
    response = get_report(analytics,request)
    return response

## Define Function to Parse Data into Lists

In [7]:
def parseResponse(response):

    dim = []
    val = []

    #Extract Data
    for report in response.get('reports', []):

        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])

        for row in rows:

            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dim.append(dimension)

            for i, values in enumerate(dateRangeValues):
                value = values.get('values')
                val.append(value)                    
    return dim, val


## Answering Business Questions

Here is a helpful resource to find dimensions and metrics you may want to pull in. 

Google Analytics Metric and Dimension Explorer: https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/

- Comment out this helper notebook when using your own data. Otherwise run to load sample data.

In [35]:
%run data/helper.ipynb

### 1. How many organic visits do we have over time?

- Let's pull in the data we need to answer this question. The main dimension is date, and we want to look at is Medium, split between Organic traffic (labelled as (none)) and Non Organic traffic (all other).

In [23]:
organic_request = {
                'viewId': VIEW_ID, 
                'dateRanges': [{'startDate': '2019-01-01', 
                                'endDate': '2019-06-01'}],
                'metrics': [{'expression': 'ga:sessions'}], 
                'dimensions': [{"name":"ga:yearweek"}], 
                'filtersExpression':'ga:medium=~(none)'
            }

nonorganic_request = {
                'viewId': VIEW_ID, 
                'dateRanges': [{'startDate': '2019-01-01', 
                                'endDate': '2019-06-01'}],
                'metrics': [{'expression': 'ga:sessions'}], 
                'dimensions': [{"name":"ga:yearweek"}], 
                'filtersExpression':'ga:medium!=~(none)'
            }


- Run functions to pull the data

In [24]:
response1 = main(organic_request)
parsed_data1 = parseResponse(response1)

response2 = main(nonorganic_request)
parsed_data2 = parseResponse(response2)

- Create a DataFrame with the parsed data and format the columns to date and integer

###### Organic Data

In [25]:
df = pd.concat([pd.DataFrame(parsed_data1[0]),pd.DataFrame(parsed_data1[1])],axis=1)
df.columns = ['Date','Organic Sessions']

df['Organic Sessions'] = df['Organic Sessions'].astype('int64')
df['Date'] = df['Date'].str.slice(0,4)+'-'+df['Date'].str.slice(4,6)

###### Non Organic Data

In [26]:
df2 = pd.concat([pd.DataFrame(parsed_data2[0]),pd.DataFrame(parsed_data2[1])],axis=1)
df2.columns = ['Date','Non-Organic Sessions']

df2['Non-Organic Sessions'] = df2['Non-Organic Sessions'].astype('int64')
df2['Date'] = df2['Date'].str.slice(0,4)+'-'+df2['Date'].str.slice(4,6)

###### Concatenate

In [27]:
df_joined = df.merge(df2,on='Date')

- Let's see what the data looks like

In [28]:
df_joined.head()

,Date,Organic Sessions,Non-Organic Sessions
0,2019-01,43,404
1,2019-02,135,453
2,2019-03,201,467
3,2019-04,121,359
4,2019-05,109,354


- Let's plot the data over time

In [29]:
df_joined.groupby(['Date'],as_index=True)['Organic Sessions','Non-Organic Sessions'].mean().\
    iplot(asFigure=True,
          yTitle = 'Sessions',
          xTitle = 'Month Year',
          title='Website Traffic over Time')

### 2. Where does search result traffic land in the website?

- Let's pull in the data we need to answer this question. We will look at the top 10 landing pages by pageviews, and analyse their bounce rates and average session duration.

In [31]:
request = {
                'viewId': VIEW_ID, 
                'dateRanges': [{'startDate': '2019-01-01', 
                                'endDate': '2019-06-01'}],
                'metrics': [{'expression': 'ga:pageviews'},
                            {'expression': 'ga:avgSessionDuration'},
                           {'expression': 'ga:bounceRate'}], 
                'dimensions': [{"name":"ga:landingPagePath"}],
                'orderBys': [{"fieldName": "ga:pageviews", "sortOrder": "DESCENDING"}], 
            }

- Run the functions to pull the data

In [32]:
response3 = main(request)
parsed_data3 = parseResponse(response3)

- Create a dataframe from the result and view the data

In [33]:
df3 = pd.concat([pd.DataFrame(parsed_data3[0]),pd.DataFrame(parsed_data3[1])],axis=1)
df3.columns = ['Landing Page','Page Views','Avg Session Duration','Bounce Rate']

df3['Page Views'] = df3['Page Views'].astype('int64')
df3['Avg Session Duration'] = df3['Avg Session Duration'].astype('float64')
df3['Bounce Rate'] = df3['Bounce Rate'].astype('float64')

df3.head()

,Landing Page,Page Views,Avg Session Duration,Bounce Rate
0,/,13688,241.194593,35.254358
1,/london/pottery,8993,87.922518,69.007264
2,/classes/seminars/discussion,5247,41.901220,80.487805
3,/london/pottery-and-ceramics,1034,48.801546,63.659794
4,/london/painting,954,103.160410,62.798635


- Graph the results

In [34]:
temp_df = df3.groupby(['Landing Page'],as_index=True)['Page Views'].mean().sort_values(ascending=False).head(10)

fig = temp_df.sort_values().\
    iplot(asFigure=True,
          kind = 'barh',
          yTitle = 'Page Views',
          xTitle = 'Landing Page',
          title='Top 10 Landing Pages')
    
fig.update_layout(
    margin=dict(l=200),
)

fig.show()

### 3. What is the distribution of session duration and bounce rate of landing pages?

- We already have the data available so let's go straight into plotting

In [20]:
fig = df3.groupby(['Landing Page'],as_index=True)[['Avg Session Duration','Bounce Rate']].mean().\
    iplot(
        asFigure=True, 
        kind ='histogram',
        bins = 40,
        subplots=True, 
        subplot_titles=True, 
        legend=False)
    
fig.show()